In [1]:
## makemore: becoming a backprop ninja

In [20]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [21]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [22]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [23]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [24]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [25]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [26]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3502, grad_fn=<NegBackward0>)

In [27]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# loss = -logprobs[range(n), Yb].mean()

dlogprobs = torch.zeros((logprobs.shape)) # can also use torch.zeros_like(logprobs)

# loss = -(a + b + c)/batch_size, therefore, derivative of each logprob wrt loss is just -1/batch_size for that index and zero for rest
dlogprobs[range(n), Yb] = -1/batch_size

# logprobs = probs.log(), therefore, derivative of each prob wrt logprobs is just 1/probs, and wrt loss is just found using chain rule
dprobs = dlogprobs / probs

# probs = counts_sum_inv * counts
# counts_sum_inv is a column vector whereas counts is a matrix so they have different shapes
# remember @ is for matmul, so we are element wise multiplying counts and dprobs
# and then we need to sum along 1st dimension to have the same shape as counts_sum_inv
# a better way to understand this is by deconstructing the forward pass into 2 operations
# 1) copying counts_sum_inv variables to make it same shape as counts
# 2) multiplying the two matrices element wise
# so to calculate derivative we can also do it for each of these operations
# 1) for element wise multiplication it is simple and we use chain rule to get counts*dprobs
# 2) for copying (broadcasting) we look it as the computational graph using each value of counts_sum_inv as an input for multiple branches
#    in the graph, so to backpropagate through a single variable we just sum the derivative produces in each of those branches 
# probs[2*2] = counts_sum_inv[2*1] * counts[2*2]
# counts_sum_inv1*counts11  counts_sum_inv1*counts12
# counts_sum_inv2*counts21  counts_sum_inv2*counts22 
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

# counts_sum_inv = counts_sum**-1
dcounts_sum = -1*counts_sum**-2 * dcounts_sum_inv

# probs = counts_sum_inv * counts , counts_sum = counts.sum(1, keepdim=True)
# the first operation derivative is simple as it is multiplication, and we are broadcasting counts_sum_inv
# the second operation is a sum along the 1st dim, which means each column has a derivative of 1
# and we can simply pass the counts_sum derivative which will be broadcast to fit the correct shape
# counts_sum[2*1] = counts[2*2].sum(1, keepdim=True)
# counts11 + counts12
# counts21 + counts22
dcounts = counts_sum_inv*dprobs + dcounts_sum

# counts = norm_logits.exp()
dnorm_logits = counts * dcounts

# norm_logits = logits - logit_maxes
# this is similar to the counts_sum_inv case since logit_maxes are broadcast in the forward pass
# therefore, we view it as the same input being used for multiple operations and in the backprop we sum the gradients
# from the operations they were involved in.
# NOTE: the reason for subtracting the max of logits from each row is for numerical stability and so the exp in the next step does not overflow
#       this means that the values of logit_maxes do not actually change the probs values and consequently they don't change the loss value
#       so logically it doesn't make sense to take a derivative through them as they should just be zero.
#       But when we break down cross_entropy into this atomic structure and get the gradients for logit_maxes, we get very small values
#       that are nearly zero but not zero. This could be due to floating point issues. However, it reaffirms our intuition that changing
#       the logit_maxes values does not affect the loss value and therefore the gradients should reflect that.
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

# norm_logits = logits - logit_maxes , logit_maxes = logits.max(1, keepdim=True).values
# in the first operation it is simple as the derivative or norm_logits wrt logits is 1
# so dnorm_logits passes straight through. Instead of using F.one_hot, we could have initialized a matrix of zeros
# and filled it in just like for dlogprobs above.
dlogits = dnorm_logits + F.one_hot(logits.max(1).indices, num_classes=logits.shape[1])*dlogit_maxes

# logits = h @ W2 + b2
dh = dlogits @ W2.T

dW2 = h.T @ dlogits

db2 = dlogits.sum(0)

# h = torch.tanh(hpreact)
dhpreact = (1 - h**2) * dh

# hpreact = bngain * bnraw + bnbias
# bngain is broadcast in the forward pass across the rows, so after getting the gradient for each row we have to sum across the rows
dbngain = (bnraw * dhpreact).sum(0)

# dnbias is also broadcast across the rows
dbnbias = dhpreact.sum(0)

# bnraw is not broadcast, but we have to broadcast bngain just like in the forward pass
dbnraw = bngain * dhpreact

# bnraw = bndiff * bnvar_inv
dbnvar_inv = (bndiff * dbnraw).sum(0)

dbndiff = dbnraw * bnvar_inv

# bnvar_inv = (bnvar + 1e-5)**-0.5
dbnvar = -0.5*((bnvar + 1e-5)**(-3/2)) * dbnvar_inv

# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
dbndiff2 = 1/(n-1)*torch.ones_like(bndiff2)*dbnvar

# bndiff2 = bndiff**2
dbndiff += (2 * bndiff * dbndiff2)

# bndiff = hprebn - bnmeani
dbnmeani = (-1 * dbndiff).sum(0, keepdim=True)

dhprebn = dbndiff.clone()

# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
dhprebn += 1/n*torch.ones_like(hprebn)*dbnmeani

# hprebn = embcat @ W1 + b1
dembcat = dhprebn @ W1.T

dW1 = embcat.T @ dhprebn

db1 = dhprebn.sum(0)

#embcat = emb.view(emb.shape[0], -1)
demb = dembcat.view(emb.shape)

#emb = C[Xb] # embed the characters into vectors
dC = torch.zeros_like(C)
for ex in range(Xb.shape[0]):
    for ix in range(len(Xb[ex])):
        dC[Xb[ex, ix]] += demb[ex, ix]


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [28]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.350170135498047 diff: -4.76837158203125e-07


In [29]:
# backward pass

# ce_loss = -1/n sum(log(Yb * logits))
# dlogits = -1/n 1/(logits*Yb)

dlogits = F.softmax(logits, dim=1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 5.122274160385132e-09


In [30]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [31]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# NOTE: calculating gradients wrt to variables that act as inputs to multiple variables in another matrix like mean and std_dev
# in manual calculations before implementation, remember to sum the contribution to the gradient to that variable from each of its outputs

dhprebn = bngain * bnvar_inv * (dhpreact - (1/(n-1))*bndiff*bnvar_inv**2*(dhpreact*bndiff).sum(0) - (1/n)*dhpreact.sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [36]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    # for p in parameters:
    #   p.grad = None
    # loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogits = F.softmax(logits, dim=1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n

    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)

    dhpreact = (1 - h**2) * dh
    dbngain = (bnraw * dhpreact).sum(0)
    dbnbias = dhpreact.sum(0)
    dhprebn = bngain * bnvar_inv * (dhpreact - (1/(n-1))*bnraw*(dhpreact*bnraw).sum(0) - (1/n)*dhpreact.sum(0))

    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    demb = dembcat.view(emb.shape)

    dC = torch.zeros_like(C)
    for ex in range(Xb.shape[0]):
        for ix in range(len(Xb[ex])):
            dC[Xb[ex, ix]] += demb[ex, ix]
    
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    # if i >= 100: # TODO: delete early breaking when you're ready to train the full net
    #   break

12297
      0/ 200000: 3.8002
  10000/ 200000: 2.1643
  20000/ 200000: 2.3834
  30000/ 200000: 2.4909
  40000/ 200000: 1.9990
  50000/ 200000: 2.3345
  60000/ 200000: 2.3498
  70000/ 200000: 2.0711
  80000/ 200000: 2.3589
  90000/ 200000: 2.1414
 100000/ 200000: 2.0009
 110000/ 200000: 2.3071
 120000/ 200000: 1.9810
 130000/ 200000: 2.5088
 140000/ 200000: 2.3184
 150000/ 200000: 2.1533
 160000/ 200000: 2.0054
 170000/ 200000: 1.8848
 180000/ 200000: 1.9741
 190000/ 200000: 1.8380


In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [38]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [39]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.070544719696045
val 2.1080093383789062


In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [40]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmahzaal.
harlin.
mrightetlanna.
sane.
mahnen.
den.
arci.
aqui.
ner.
kia.
chaiivon.
leigh.
ham.
join.
quinthorline.
livani.
watell.
dearisia.
kaellinsley.
dae.
